In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [3]:
movies=pd.read_csv('/content/drive/MyDrive/MOVIE RECOMMENDATION/movies.csv')
ratings=pd.read_csv('/content/drive/MyDrive/MOVIE RECOMMENDATION/ratings.csv')

In [4]:
genres=[]
for genre in movies.genres:

    x=genre.split('|')
    for i in x:
         if i not in genres:
            genres.append(str(i))
genres=str(genres)
movie_title=[]
for title in movies.title:
    movie_title.append(title[0:-7])
movie_title=str(movie_title)

In [5]:
df=pd.merge(ratings,movies, how='left',on='movieId')
df.head()

,userId,movieId,rating,timestamp,title,genres
0,1,16,4.0,1217897793,Casino (1995),Crime|Drama
1,1,24,1.5,1217895807,Powder (1995),Drama|Sci-Fi
2,1,32,4.0,1217896246,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller
3,1,47,4.0,1217896556,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,4.0,1217896523,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


In [6]:
df1=df.groupby(['title'])[['rating']].sum()
high_rated=df1.nlargest(20,'rating')
high_rated.head()

,rating
title,
"Shawshank Redemption, The (1994)",1372.0
Pulp Fiction (1994),1352.0
Forrest Gump (1994),1287.0
"Silence of the Lambs, The (1991)",1216.5
Star Wars: Episode IV - A New Hope (1977),1143.5


In [7]:
df2=df.groupby('title')[['rating']].count()
rating_count_20=df2.nlargest(20,'rating')
rating_count_20.head()

,rating
title,
Pulp Fiction (1994),325
Forrest Gump (1994),311
"Shawshank Redemption, The (1994)",308
Jurassic Park (1993),294
"Silence of the Lambs, The (1991)",290


In [8]:
cv=TfidfVectorizer()
tfidf_matrix=cv.fit_transform(movies['genres'])

In [9]:
movie_user = df.pivot_table(index='userId',columns='title',values='rating')
movie_user.head()

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Til There Was You (1997),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...And Justice for All (1979),10 (1979),...,[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),a/k/a Tommy Chong (2005),eXistenZ (1999),loudQUIETloud: A Film About the Pixies (2006),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Suppose a user wants to watch a movie similar to Toy Story (1995) then we can recommend the user by calculating the cosine similarity between Toy Story and other movies.

In [10]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [11]:
indices=pd.Series(movies.index,index=movies['title'])
titles=movies['title']
def recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:21]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

In [12]:
recommendations('(500) Days of Summer (2009)')

10                  American President, The (1995)
48                         Mighty Aphrodite (1995)
53               Postman, The (Postino, Il) (1994)
86                          Beautiful Girls (1996)
168                 Something to Talk About (1995)
195                        Don Juan DeMarco (1995)
203    Eat Drink Man Woman (Yin shi nan nu) (1994)
250                           Nobody's Fool (1994)
311                        Corrina, Corrina (1994)
320                  It Could Happen to You (1994)
331                           Reality Bites (1994)
482                    Sleepless in Seattle (1993)
485                              Son in Law (1993)
631                    Month by the Lake, A (1995)
659                     Walking and Talking (1996)
682                                    Emma (1996)
693                                 Tin Cup (1996)
715                           Twelfth Night (1996)
719                 Philadelphia Story, The (1940)
730                          Ap

In [14]:
recommendations('\'Til There Was You (1997)')

24                            Leaving Las Vegas (1995)
27                                   Persuasion (1995)
34                                   Carrington (1995)
43                How to Make an American Quilt (1995)
46                        When Night Is Falling (1995)
68                                 Bed of Roses (1996)
76     Once Upon a Time... When We Were Colored (1995)
78                           Angels and Insects (1995)
95               Bridges of Madison County, The (1995)
114                           Frankie Starlight (1995)
119                       Up Close and Personal (1996)
154                                    Mad Love (1995)
157                     Moonlight and Valentino (1995)
165                         Scarlet Letter, The (1995)
174                               Total Eclipse (1995)
179                       Walk in the Clouds, A (1995)
187                              Before Sunrise (1995)
193                           Circle of Friends (1995)
219       

In [15]:
recommendations('Shawshank Redemption, The (1994)')

29     Shanghai Triad (Yao a yao yao dao waipo qiao) ...
35                               Dead Man Walking (1995)
89                               Hate (Haine, La) (1995)
105                Young Poisoner's Handbook, The (1995)
215                              The Glass Shield (1994)
217                            Heavenly Creatures (1994)
237                                 Little Odessa (1994)
252                              New Jersey Drive (1995)
255                            Once Were Warriors (1994)
279                     Shawshank Redemption, The (1994)
350                                 Jason's Lyric (1994)
358                                 Above the Rim (1994)
379                                 Carlito's Way (1993)
662                              American Buffalo (1996)
691                               Cyclo (Xich lo) (1995)
695                                Godfather, The (1972)
780                          Angel on My Shoulder (1946)
881                            